## Lab Assignment Three: Extending Logistic Regression

#### Luis Garduno

### 1. Overview & Preparation

Explain Here

-------------------------------------
Dataset :

Question Of Interest :

### 2. Modeling
### 3. Deployment
### 4. Logistic Regression Optimization
#### Reference